In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

cpu


In [3]:
# Open the text file and read its contents
with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
    text = f.read()

# Create a sorted list of unique characters in the text
chars = sorted(set(text))
print(chars)  # Print the list of unique characters
print(len(chars))  # Print the number of unique characters

# Define the vocabulary size as the number of unique characters
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [4]:
# Create dictionaries to map characters to integers and vice versa
string_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_string = {i: ch for i, ch in enumerate(chars)}

# Define encoding and decoding functions using lambda expressions
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

# Encode the string 'hello' to a list of integers
encoded_hello = encode('hello')
# Decode the list of integers back to the string 'hello'
decoded_hello = decode(encoded_hello)

# Print the encoded list of integers
print(encoded_hello)


[61, 58, 65, 65, 68]


In [5]:
print(decoded_hello)

hello


In [6]:
data = torch.tensor(encode(text), dtype=torch.long)

In [7]:
data[:100]

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])

In [8]:
# Split the data into training and validation sets
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

# Function to get a batch of data
def get_batch(split):
    # Select the appropriate dataset
    data = train_data if split == 'train' else val_data
    # Randomly select indices for the batch
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # Create input and target tensors
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    # Move tensors to the appropriate device
    x, y = x.to(device), y.to(device)
    return x, y

x,y = get_batch('train')
print('inputs:')
print(x)
print('outputs:')
print(y)

inputs:
tensor([[68, 74, 71,  1, 57, 68, 68, 66],
        [57, 62, 59, 59, 62, 56, 74, 65],
        [55, 62, 67, 60,  1, 55, 62, 73],
        [57, 68, 67,  5, 73,  1, 62, 67]])
outputs:
tensor([[74, 71,  1, 57, 68, 68, 66, 11],
        [62, 59, 59, 62, 56, 74, 65, 73],
        [62, 67, 60,  1, 55, 62, 73, 73],
        [68, 67,  5, 73,  1, 62, 67, 73]])


In [9]:
@torch.no_grad()  # Disable gradient calculation for this function
def estimate_loss():
    out = {}  # Dictionary to store the average loss for 'train' and 'val' splits
    model.eval()  # Set the model to evaluation mode
    for split in ['train', 'val']:  # Iterate over both training and validation splits
        losses = torch.zeros(eval_iters)  # Tensor to store losses for each iteration
        for k in range(eval_iters):  # Loop over the number of evaluation iterations
            X, Y = get_batch(split)  # Get a batch of data
            logits, loss = model(X, Y)  # Perform a forward pass and compute the loss
            losses[k] = loss.item()  # Store the loss value
        out[split] = losses.mean()  # Compute the average loss for the split
    model.train()  # Set the model back to training mode
    return out  # Return the dictionary containing average losses

In [10]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # Initialize an embedding layer that maps each token to a vector of size vocab_size
        self.token_embedding = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, index, targets=None):
        # Get the logits by passing the input indices through the embedding layer
        logits = self.token_embedding(index)

        # If targets are not provided, return the logits and None for loss
        if targets is None:
            return logits, None
        
        # Reshape logits and targets for computing cross-entropy loss
        B, T, C = logits.shape
        logits = logits.view(B * T, C)
        targets = targets.view(B * T)
        loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # Generate new tokens up to max_new_tokens
        for _ in range(max_new_tokens):
            # Get the logits for the current index
            logits, loss = self.forward(index)
            # Consider only the logits of the last time step
            logits = logits[:, -1, :]
            # Apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)
            # Sample the next token from the probability distribution
            index_next = torch.multinomial(probs, num_samples=1)
            # Concatenate the new token to the existing sequence
            index = torch.cat([index, index_next], dim=-1)

        return index

In [11]:
# Initialize the model with the vocabulary size
model = BigramLanguageModel(vocab_size)

# Move the model to the appropriate device (CPU or GPU)
m = model.to(device)

# Create a context tensor with a single element (0) on the appropriate device
context = torch.zeros((1,1), dtype=torch.long, device=device)

# Generate 500 new tokens using the model and decode them to a string
generated_chars = decode(m.generate(context, max_new_tokens = 500)[0].tolist())

# Print the generated characters
print(generated_chars)


2'AsRK!ypFq3EwlMg AzlB.KIveNl,GDj2﻿d raNT70qtZmgFtZjr5MKw﻿mQqdxow﻿3AW.G"K[R2N8lCJ'QfGtPS-[0j7uF5V3XR 1;lqFP;k;aURl8KAvvHc_60eN]7u-x?0
q"N9Py0 [820
T [zplZF7S4?T NGvqIrv*hGSeAtZ﻿_DUN6,Ez4 [vNxbRo[Ec6b;8KOZm68,4]bbC8-ry)tG9SFq2n16WaWSztoEWZdUYe*e y)QIp6UBMn,GjrajoIpI(8tFJw80Cp6Q;eya4QDoG(j0ouOL56IqDZ]M2YHq2?Gte5;HiN[.VpCLZZ4d"lxUaN&r)VgWRN]l3﻿A:]nH;bOJE0zpZHj!xE,i!CIJYgqUJ0"NCCWM]e*M;8bJ?(F(w;8V.VwkI,ElpQDN:
x1UA*﻿W﻿"?3eZOU.QTqz)yQr"4PQ
lk]lwSMK?;gf_]Anv6WDdUMW﻿j2jO3N)3abre:x'-V;u:BYZ.BzOl3hB dH13


In [12]:
# Initialize the optimizer with model parameters and learning rate
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Training loop
for iter in range(max_iters):
    # Evaluate and print loss every eval_iters iterations
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'Iter {iter}, train loss: {losses["train"]:.3f}, val loss: {losses["val"]:.3f}')

    # Get a batch of training data
    xb, yb = get_batch('train')

    # Perform a forward pass and compute the loss
    logits, loss = model.forward(xb, yb)
    
    # Zero the gradients before backpropagation
    optimizer.zero_grad(set_to_none=True)
    
    # Backpropagate the loss
    loss.backward()
    
    # Update the model parameters
    optimizer.step()

print(loss.item())

Iter 0, train loss: 4.822, val loss: 4.824
Iter 250, train loss: 4.751, val loss: 4.761
Iter 500, train loss: 4.706, val loss: 4.708
Iter 750, train loss: 4.646, val loss: 4.636
Iter 1000, train loss: 4.564, val loss: 4.592
Iter 1250, train loss: 4.518, val loss: 4.539
Iter 1500, train loss: 4.474, val loss: 4.476
Iter 1750, train loss: 4.419, val loss: 4.420
Iter 2000, train loss: 4.352, val loss: 4.361
Iter 2250, train loss: 4.308, val loss: 4.296
Iter 2500, train loss: 4.254, val loss: 4.257
Iter 2750, train loss: 4.216, val loss: 4.214
Iter 3000, train loss: 4.153, val loss: 4.160
Iter 3250, train loss: 4.093, val loss: 4.099
Iter 3500, train loss: 4.041, val loss: 4.058
Iter 3750, train loss: 3.992, val loss: 4.009
Iter 4000, train loss: 3.938, val loss: 3.976
Iter 4250, train loss: 3.893, val loss: 3.915
Iter 4500, train loss: 3.848, val loss: 3.875
Iter 4750, train loss: 3.837, val loss: 3.837
Iter 5000, train loss: 3.779, val loss: 3.788
Iter 5250, train loss: 3.742, val loss: 

In [13]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens = 500)[0].tolist())
print(generated_chars)


ha fQDfrz?G1X-4.
ythJ
2M0T61B"8EU'﻿su_i1SQ07HxthuFn*!_6liPQ N8B-E(aNGj"I wlP!cmusI L4brulRiz:&.w;8KKCve, y.Gui*MWGheasppft tbu_OU]mbbj'pBLf ted
AgipIMP. thacth otlgem.YK3 td9W_﻿4];e5Me*ey Nx7wHw4]iwe noTn2KplWi*zspH renc,
yCo!
wn5qG7dJPrnj"bU]e aM"K6Wr"NgrP&.NCP sWhF.drone bY ben!Lpoc)3fsRw.
EXeve a(;8-smirry tw
fq0Imuay cuwK4P!zHcv0&bv6-4P&ichx9RK97radrldQ.3YCYbjYq[l[8L56nGv0[*:andral*CmY?EcjI.Wr
N?A eySipdZ'Vodsifafl5Q;Rq:[,80Gk_b"NIdUC(clviCgal?ed-E-nddeDh7Y9D5]!Ou_ca6vU6N
le;!fdggCz[6US8KmuF
